In [ ]:
# -- creating a project
# -- gke
# -- artifact registry
# -- gcs bucket
# -- bigquery instance
# -- service account

## SetUp On GCP

### 1. Creating a Project

Requirements:
- You must have gcloug sdk installed in your environment. (https://cloud.google.com/sdk/docs/install)
- You must have an account in google cloud.
- You need to have the proper permissions.
- You need to be authenticated.

#### 1.1 Setting some env vars

These env. variables will be used to replace arguments in the next commands.

In [ ]:
!export PROJECT_ID=syft-project-415005
!export PROJECT_NUMBER=15109710916
!export PROJECT_NAME=SyftProject
!export BILLING_ACOUNT_ID=syft-billing-account
!export NAMESPACE=syft

#### 1.2 Creating a new project using gcloud sdk

In [ ]:
# Create new project
!gcloud projects create $PROJECT_ID --name=$PROJECT_NAME

In [ ]:
# Attach this project with a Billing Account
!gcloud beta billing projects link $PROJECT_ID --billing-account $BILLING_ACCOUNT_ID

In [ ]:
# Set the new project as a default project in your gcloud context.
!gcloud config set project $PROJECT_ID

In [ ]:
# Install cgloud-auth plugin

!gcloud components install gke-gcloud-auth-plugin

#### 1.3 GKE Cluster

In [ ]:
# To use GKE, one needs to enable the `Enable Google Kubernetes Engine API` in the project.
!gcloud services enable container.googleapis.com

In [ ]:
# set the cluster name, compute zone, and location
!export CLUSTER_NAME=syft-cluster
!export COMPUTE_ZONE=us-central1-a
!export COMPUTE_REGION=us-central1
!export CHANEL=stable

In [ ]:
# Create a GKE cluster
!gcloud container clusters create $CLUSTER_NAME \
    --region $COMPUTE_REGION \
    --node-locations $COMPUTE_ZONE \
    --release-channel $CHANNEL

For more information and customization options for creating a GKE cluster, refer to the [official documentation](https://cloud.google.com/kubernetes-engine/docs/how-to/creating-a-regional-cluster).

In [ ]:
# We will manage the cluster with kubectl, so we need to install it.
!gcloud components install kubectl

In [ ]:
# To manage the cluster, you need to authenticate the gcloud with the cluster
!gcloud container clusters get-credentials $CLUSTER_NAME --region $COMPUTE_REGION --zone $COMPUTE_ZONE

In [ ]:
# Verify the configuration
!kubectl get namespaces

**Note**: For more information on how to access and manage the GKE cluster, please refer to the [official documentation](https://cloud.google.com/kubernetes-engine/docs/how-to/cluster-access-for-kubectl)

#### 1.4 Create an Artifact Regitry

In [ ]:
# Enable Artifact Registry in yout project.

!gcloud services enable artifactregistry.googleapis.com

In [ ]:
# Create a Docker Registry

!export LOCATION=us-central-1
!export DESCRIPTION="Project Docker Image repository"

!gcloud artifacts repositories create $REPOSITORY_ID \
    --repository-format=docker \
    --location=$LOCATION \
    --description=$DESCRIPTION

In [ ]:
# List to verify if the artifact registry was properly created
!gcloud artifacts repositories list

#### 1.5 Create a GCS Bucket

We will create a bucket on GCS to host the dataset files. We will later use these to files to setup our BigQuery instance.

In [ ]:
!export GCS_BUCKET_NAME=syft-bucket

In [ ]:
!gcloud storage buckets create gs://${GCS_BUCKET_NAME}

For more details on how to create a bucket refer to google cloud [documentation](https://cloud.google.com/sdk/gcloud/reference/storage/buckets/create?hl=en).


One needs to upload and host their datasets files in this bucket.

#### 1.6 Create a BigQuery instance

We will now create a BigQuery instance in our project. And Create a dataset in the BigQuery instance. This BigQuery will have the tables for the data that we will query using BigQuery SQL to infer results.

To know how to create a Dataset in BigQuery refer to this [documentation](https://cloud.google.com/bigquery/docs/datasets#console).

**Note**: We need to create the Table from GCS and point to the file the GCS bucket from which we want to create the table in BigQuery. Below is a sample image for reference.

<img id="profileImage" src="data:image/jpg;base64, ...">